<a href="https://colab.research.google.com/github/Maks-Shashkov/detailed_math/blob/main/Discrete_Math2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Построить схему алфавитного кодирования для двухбуквенных сочетаний методом
Шеннона-Фано. Найти среднюю длину элементарного кода, эффективность сжатия,
сравнить с результатами для однобуквенных сочетаний. Закодировать текст. Декодировать
текст.

Код использует метод Шеннона-Фано для построения алфавитного кода для заданного текста. Он начинается с расчета вероятностей появления каждого символа (или сочетания символов, если речь идет о двухбуквенных сочетаниях). Затем эти вероятности используются для построения дерева Хаффмана. В данном случае модифицированную версию алгоритма Шеннона-Фано, которая разбивает узлы на две группы почти равной вероятности.

Дерево Хаффмана используется для построения алфавитного кода, где каждому символу (или сочетанию символов) присваивается уникальный код. Для этого мы обходим дерево и присваиваем "0" всем узлам, которые находятся на левой ветке, и "1" всем узлам на правой ветке. Код для каждого символа (или сочетания символов) составляется путем объединения всех "0" и "1", которые соответствуют узлам на пути от корня до листьев.

Средняя длина элементарного кода определяется как сумма произведений длины кода на вероятность для каждого символа (или сочетания символов). Эффективность сжатия определяется как отношение количества битов, необходимых для представления исходного текста, к количеству битов, необходимых для представления закодированного текста.

Закодированный текст получается путем замены каждого символа (или сочетания символов) его кодом. Декодирование текста осуществляется путем обратной замены кодов на символы (или сочетания символов) с использованием таблицы кодов

In [ ]:
class Node:
    def __init__(self, symbol=None, probability=0.0):
        self.symbol = symbol
        self.probability = probability
        self.left = None
        self.right = None

def build_tree(symbols, probabilities):
    nodes = [Node(symbol=symbol, probability=probability) for symbol, probability in zip(symbols, probabilities)]
    while len(nodes) > 1:
        nodes = sorted(nodes, key=lambda node: node.probability, reverse=True)
        middle = len(nodes) // 2
        left_nodes = nodes[:middle]
        right_nodes = nodes[middle:]
        parent = Node(probability=sum(node.probability for node in nodes))
        parent.left = build_tree([node.symbol for node in left_nodes], [node.probability for node in left_nodes])
        parent.right = build_tree([node.symbol for node in right_nodes], [node.probability for node in right_nodes])
        nodes = [parent]
    return nodes[0]

def assign_codes(node, code=''):
    if node.symbol is not None:
        codes[node.symbol] = code
    else:
        assign_codes(node.left, code + '0')
        assign_codes(node.right, code + '1')

# Пример использования:
text = "ABBCCCDDDDEEEEE"
symbols = sorted(set([text[i:i+2] for i in range(0, len(text), 2)]))
probabilities = [text.count(symbol) / (len(text) / 2) for symbol in symbols]

root = build_tree(symbols, probabilities)
codes = {}
assign_codes(root)

average_code_length = sum([len(codes[symbol]) * text.count(symbol) for symbol in symbols]) / len(text)
compression_ratio = len(text) * 8 / (len(symbols) * average_code_length)

print("Символы:", symbols)
print("Вероятности:", probabilities)
print("Коды:", codes)
print("Средняя длина кода:", average_code_length)
print("Эффективность сжатия:", compression_ratio)

# Кодирование текста
encoded_text = ''.join([codes[text[i:i+2]] for i in range(0, len(text), 2)])
print("Закодированный текст:", encoded_text)

# Декодирование текста
decoded_text = ''
i = 0
while i < len(encoded_text):
    found = False
    for symbol in symbols:
        if encoded_text.startswith(codes[symbol], i):
            decoded_text += symbol
            i += len(codes[symbol])
            found = True
            break
    if not found:
        break

print("Декодированный текст:", decoded_text)

Символы: ['AB', 'BC', 'CC', 'DD', 'E', 'EE']
Вероятности: [0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.26666666666666666, 0.6666666666666666, 0.26666666666666666]
Коды: {'E': '00', 'DD': '010', 'EE': '011', 'AB': '10', 'BC': '110', 'CC': '111'}
Средняя длина кода: 2.0
Эффективность сжатия: 10.0
Закодированный текст: 1011011101001001101100
Декодированный текст: ABBCCCDDDDEEEEE


In [1]:
import numpy as np
import math

# Функция для вычисления энтропии текста
def calculate_entropy(text):
    char_count = len(text)
    freq_dict = {}
    for char in text:
        if char in freq_dict:
            freq_dict[char] += 1
        else:
            freq_dict[char] = 1

    entropy = 0
    for freq in freq_dict.values():
        probability = freq / char_count
        entropy -= probability * math.log2(probability)

    return entropy / char_count

# Функция для построения алфавитного кодирования методом Шеннона-Фано
def shannon_fano_coding(text):
    char_count = len(text)
    freq_dict = {}
    for char in text:
        if char in freq_dict:
            freq_dict[char] += 1
        else:
            freq_dict[char] = 1

    sorted_freq = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)

    def build_code(prefix, symbols):
        if len(symbols) == 1:
            return {symbols[0][0]: prefix}
        else:
            mid = len(symbols) // 2
            left_symbols = symbols[:mid]
            right_symbols = symbols[mid:]
            left_code = build_code(prefix + "0", left_symbols)
            right_code = build_code(prefix + "1", right_symbols)
            return {**left_code, **right_code}

    code_dict = build_code("", sorted_freq)

    return code_dict

# Функция для кодирования текста с использованием алфавитного кодирования
def encode_text(text, code_dict):
    encoded_text = ""
    for char in text:
        encoded_text += code_dict[char]
    return encoded_text

# Функция для декодирования текста с использованием алфавитного кодирования
def decode_text(encoded_text, code_dict):
    reverse_code_dict = {v: k for k, v in code_dict.items()}
    decoded_text = ""
    current_code = ""
    for bit in encoded_text:
        current_code += bit
        if current_code in reverse_code_dict:
            decoded_text += reverse_code_dict[current_code]
            current_code = ""
    return decoded_text

# Текст для анализа
text = "аааааабббб11123"

# Однобуквенные сочетания
entropy = calculate_entropy(text)
code_dict = shannon_fano_coding(text)
encoded_text = encode_text(text, code_dict)
decoded_text = decode_text(encoded_text, code_dict)

print("Однобуквенные сочетания:")
print("Энтропия на одну букву:", entropy)
print("Длина кода при равномерном кодировании:", math.ceil(math.log2(len(code_dict))))
print("Избыточность:", 1 - (entropy / math.ceil(math.log2(len(code_dict)))))
print("Схема алфавитного кодирования:", code_dict)
print("Закодированный текст:", encoded_text)
print("Декодированный текст:", decoded_text)

# Двухбуквенные сочетания
text = "00011101"

entropy = calculate_entropy(text)
code_dict = shannon_fano_coding(text)
encoded_text = encode_text(text, code_dict)
decoded_text = decode_text(encoded_text, code_dict)

print("\nДвухбуквенные сочетания:")
print("Энтропия на одну букву:", entropy)
print("Длина кода при равномерном кодировании:", math.ceil(math.log2(len(code_dict))))
print("Избыточность:", 1 - (entropy / math.ceil(math.log2(len(code_dict)))))
print("Схема алфавитного кодирования:", code_dict)
print("Закодированный текст:", encoded_text)
print("Декодированный текст:", decoded_text)

Однобуквенные сочетания:
Энтропия на одну букву: 0.13483865078949944
Длина кода при равномерном кодировании: 3
Избыточность: 0.9550537830701669
Схема алфавитного кодирования: {'а': '00', 'б': '01', '1': '10', '2': '110', '3': '111'}
Закодированный текст: 00000000000001010101101010110111
Декодированный текст: аааааабббб11123

Двухбуквенные сочетания:
Энтропия на одну букву: 0.125
Длина кода при равномерном кодировании: 1
Избыточность: 0.875
Схема алфавитного кодирования: {'0': '0', '1': '1'}
Закодированный текст: 00011101
Декодированный текст: 00011101
